In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time

In [49]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_mac64.zip
Driver has been saved in cache [/Users/curtissmith/.wdm/drivers/chromedriver/mac64/93.0.4577.63]


In [52]:
# Visit the Beer Advocate Top 250 page
url = 'https://www.beeradvocate.com/beer/top-styles/'
browser.visit(url)

In [53]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
styles_home = soup(html, 'html.parser')

In [54]:
# Find all anchor tags
all_hrefs = styles_home.find_all('a')

In [55]:
# Get hrefs for style links
x = [item.get('href') for item in all_hrefs if "/beer/top-styles/" in str(item)]

In [56]:
styles_hrefs = []
error_hrefs = ['<a href="/beer/top-styles/">Beers by Style</a>',
              '<a href="/beer/top-styles/#XenForo"><i class="uix_icon uix_icon-jumpToTop"></i> <span class="uix_hide">Top</span></a>',
              '<a href="/beer/top-styles/">Beers by Style</a>',
              '<a href="/beer/top-styles/">Styles</a>']
for item in all_hrefs:
    if "/beer/top-styles/" in str(item):
        if str(item) not in error_hrefs:
            styles_hrefs.append(item.get('href'))

In [61]:
start_time = time.time()

count = 0
num_skipped = 0
errors = []
beer_info = []
mininum_num_reviews = 100


browser.visit(url)
styles_home = soup(html, 'html.parser')


#iterate through style links
for style_link in styles_hrefs:
    
    try:
        
        # Setting url variable to navigate back to
        style_url = f"https://www.beeradvocate.com{style_link}"
        
        # navigate to link
        browser.click_link_by_href(style_link)

        # Parse the beer html with soup
        html = browser.html
        style_profile = soup(html, 'html.parser')
        
        # find all a tags
        all_a = style_profile.find_all('a')

        # select only relevant link (those with "beer/profile" in link)
        links = [x for x in all_a if '/beer/profile/' in str(x)]

        # iterate and get links to beers - even indices are links to beers, odd are links to brewery
        beer_a_tags = []
        for index, item in enumerate(links):
            if index % 2 == 0:
                beer_a_tags.append(item)

        # get href url for navigating to beer webpage
        beer_href = [x.get('href') for x in beer_a_tags]
                
        #iterate through links
        for link in beer_href:
            
            count += 1
            if count % 50 == 0:
                length = (time.time() - start_time)
                print(f'Elapsed Time: {length} seconds / {length/60} minutes')
                print(f'Parsed {count} beers')
                print(f'Added {len(beer_info)} beers with >= {mininum_num_reviews}')

            # navigate to link
            browser.click_link_by_href(link)

            # Parse the beer html with soup
            html = browser.html
            beer_soup = soup(html, 'html.parser')
            
            try:

                reviews = []

                spans = beer_soup.find_all('span', class_='Tooltip')
                num_reviews = spans[10].text
                num_reviews = int(num_reviews.replace(",", ""))

                # Only have beers with > 100 reviews
                if num_reviews < mininum_num_reviews:
#                     print(num_reviews)
                    num_skipped += 1
    
                    browser.visit(style_url)

                    # Parse the beer html with soup
                    html = browser.html
                    style_profile = soup(html, 'html.parser')
    
                    continue

                sorting_links = [f"{link}?sort=top",
                                f"{link}?view=beer&sort=top&start=25",
                                f"{link}?view=beer&sort=top&start=50",
                                f"{link}?view=beer&sort=top&start=100"]


                for item in sorting_links:

                    browser.click_link_by_href(item)

                    html = browser.html
                    beer_soup = soup(html, 'html.parser')

                    # Get beer and brewery
                    header = beer_soup.find('div', class_='titleBar')
                    brewery = header.find("span").text
                    beer_name = header.text.replace(brewery, "")
                    beer_name = beer_name.replace('\n', "")

                    # get ABV and rating
                    spans = beer_soup.find_all('span', class_='Tooltip')
                    abv = spans[2].text
                    rating = spans[6].text

                    # get style
                    a = beer_soup.find_all('a', class_='Tooltip')
                    style = a[4].text

                    # get reviews
                    review_div = beer_soup.find_all('div', {"id": "rating_fullview_content_2"})
                    for item in review_div:
                        reviews.append(item.text.split("%")[1])

                # create dictionary and add to list
                beer_info.append({"beer_name": beer_name,
                                  "beer_style": style,
                                  "brewery": brewery,
                                  "abv": abv,
                                  "rating": rating,
                                  "reviews": reviews})

                #print(len(reviews))

                browser.visit(style_url)

                # Parse the beer html with soup
                html = browser.html
                style_profile = soup(html, 'html.parser')                       
            
            except Exception as exception:
                print(exception)
                errors.append(beer_name)
                print(f'Current Error Count: {len(errors)}')
                
                
                if browser.url == 'https://www.google.com/':
                    break
                
                browser.visit(style_url)

                # Parse the beer html with soup
                html = browser.html
                style_profile = soup(html, 'html.parser') 
                        
        
    except Exception as e:
        print(e)
        
        if browser.url == 'https://www.google.com/':
            break
        
        browser.visit(url)

        # Parse the beer html with soup
        browser.visit(url)
        styles_home = soup(html, 'html.parser')
  
        
length = (time.time() - start_time)
print(f'Script ran in {length} seconds / {length/60} minutes')
print(f'Skipped {num_skipped} beers')
print(f'There were {errors} errors')

Elapsed Time: 186.24668383598328 seconds / 3.104111397266388 minutes
Parsed 50 beers
Added 19 beers with >= 100
invalid literal for int() with base 10: 'Ratings:'
Current Error Count: 1
Elapsed Time: 291.82695865631104 seconds / 4.863782644271851 minutes
Parsed 100 beers
Added 23 beers with >= 100
no elements could be found with link by href "/beer/top-styles/36/"
invalid literal for int() with base 10: 'Ratings:'
Current Error Count: 2
Elapsed Time: 420.50592589378357 seconds / 7.008432098229727 minutes
Parsed 150 beers
Added 31 beers with >= 100
Elapsed Time: 524.5982098579407 seconds / 8.743303497632345 minutes
Parsed 200 beers
Added 32 beers with >= 100
no elements could be found with link by href "/beer/top-styles/32/"
Elapsed Time: 697.9682788848877 seconds / 11.632804648081462 minutes
Parsed 250 beers
Added 45 beers with >= 100
no elements could be found with link by href "/beer/top-styles/86/"
Elapsed Time: 812.289675951004 seconds / 13.538161265850068 minutes
Parsed 300 beers


Elapsed Time: 5522.901613712311 seconds / 92.04836022853851 minutes
Parsed 2300 beers
Added 321 beers with >= 100
invalid literal for int() with base 10: 'Ratings:'
Current Error Count: 20
Elapsed Time: 5634.521743774414 seconds / 93.90869572957357 minutes
Parsed 2350 beers
Added 325 beers with >= 100
no elements could be found with link by href "/beer/top-styles/43/"
Elapsed Time: 5754.777858734131 seconds / 95.91296431223552 minutes
Parsed 2400 beers
Added 330 beers with >= 100
no elements could be found with link by href "/beer/top-styles/21/"
Elapsed Time: 5872.3382477760315 seconds / 97.87230412960052 minutes
Parsed 2450 beers
Added 337 beers with >= 100
invalid literal for int() with base 10: 'Ratings:'
Current Error Count: 21
no elements could be found with link by href "/beer/profile/8/91718/?view=beer&sort=top&start=100"
Current Error Count: 22
invalid literal for int() with base 10: 'Ratings:'
Current Error Count: 23
no elements could be found with link by href "/beer/top-sty

KeyboardInterrupt: 